## Running a dynamic stochastic discrete continuous choice model 

Import the necessary libraries

In [1]:
# packages needed
import jax.numpy as jnp
import numpy as np
from jax import config

# import model_funcs
from model_funcs import (
    budget_constraint_exp,
    create_final_period_utility_function_dict,
    create_state_space_function_dict,
    create_utility_function_dict,
    job_offer,
    prob_survival,
)

import dcegm

config.update("jax_enable_x64", True)

Set params, model config, model specs and set up state transition funcs

Set model config

In [5]:
params = {
    "discount_factor": 0.9523809523809523,
    # disutility of work
    "delta": 0.35,
    # CRRA coefficient
    "rho": 1.95,
    # labor income coefficients
    "constant": 0.75,
    "exp": 0.04,
    "exp_squared": -0.0002,
    # Shock parameters of income
    "income_shock_std": 0.35,
    "income_shock_mean": 0.0,
    "taste_shock_scale": 0.2,
    "interest_rate": 0.05,
    "consumption_floor": 0.001,
}


n_periods = 20
n_choices = 3


model_config = {
    "min_period_batch_segments": [5, 12],
    "n_periods": n_periods,
    "choices": np.arange(n_choices, dtype=int),
    "deterministic_states": {
        "already_retired": np.arange(2, dtype=int),
    },
    "continuous_states": {
        "assets_end_of_period": jnp.arange(0, 100, 5, dtype=float),
        "experience": jnp.linspace(0, 1, 7, dtype=float),
    },
    "stochastic_states": {
        "job_offer": [0, 1],
        "survival": [0, 1],
    },
    "n_quad_points": 5,
}

stochastic_state_transitions = {
    "job_offer": job_offer,
    "survival": prob_survival,
}

model_specs = {
    "n_periods": n_periods,
    "n_choices": 3,
    "min_ret_period": 5,
    "max_ret_period": 10,
    "fresh_bonus": 0.1,
    "exp_scale": 20,
}

Set up, solve, and simulate the model

In [6]:
model = dcegm.setup_model(
    model_specs=model_specs,
    model_config=model_config,
    utility_functions=create_utility_function_dict(),
    utility_functions_final_period=create_final_period_utility_function_dict(),
    state_space_functions=create_state_space_function_dict(),
    budget_constraint=budget_constraint_exp,
    stochastic_states_transitions=stochastic_state_transitions,
)

# Solve the model
model_solved = model.solve(params)

# Simulate the model
n_agents = 1_000
states_initial = {
    "period": jnp.zeros(n_agents),
    "lagged_choice": jnp.ones(n_agents) * 2,  # all agents start as full time workers
    "already_retired": jnp.zeros(n_agents),
    "job_offer": jnp.ones(n_agents),
    "survival": jnp.ones(n_agents),
    "experience": jnp.ones(n_agents),
    "assets_begin_of_period": jnp.ones(n_agents) * 10,
}

model_solved.simulate(states_initial=states_initial, seed=42)

Starting state space creation
State space created.

Starting state-choice space creation and child state mapping.
State, state-choice and child state mapping created.

Start creating batches for the model.
The batch size of the backwards induction is  1
The batch size of the backwards induction is  2
The batch size of the backwards induction is  6
Model setup complete.



already_retired  assets_begin_of_period  choice   consumption  \
period agent                                                                  
0      0                    0               10.000000       2  3.455242e-21   
       1                    0               10.000000       2  3.455242e-21   
       2                    0               10.000000       2  3.455242e-21   
       3                    0               10.000000       2  3.455242e-21   
       4                    0               10.000000       2  3.455242e-21   
...                       ...                     ...     ...           ...   
19     995                  1                3.304585       0  3.304585e+00   
       996                  1                2.535658       0  2.535658e+00   
       997                  1                5.731518       0  5.731518e+00   
       998                  1                2.092372       0  2.092372e+00   
       999                  1                8.766500       0  8.766500e+00   

              experience  income_shock  job_offer  lagged_choice  savings  \
period agent                                                                
0      0            1.00     -0.444792          1              2     10.0   
       1            1.00     -0.079140          1              2     10.0   
       2            1.00     -0.270214          1              2     10.0   
       3            1.00      0.091201          1              2     10.0   
       4            1.00     -0.041055          1              2     10.0   
...                  ...           ...        ...            ...      ...   
19     995          1.05      0.000000          0              0      0.0   
       996          1.05      0.000000          1              0      0.0   
       997          1.40      0.000000          0              0      0.0   
       998          1.10      0.000000          0              0      0.0   
       999          1.35      0.000000          1              0      0.0   

              survival    utility     value_max  taste_shocks_0  \
period agent                                                      
0      0             1 -47.814394  4.870858e+22       -0.164279   
       1             1 -47.814394  4.870858e+22        0.213933   
       2             1 -47.814394  4.870858e+22       -0.330836   
       3             1 -47.814394  4.870858e+22       -0.122751   
       4             1 -47.814394  4.870858e+22        0.010352   
...                ...        ...           ...             ...   
19     995           1   1.195311  1.258769e+00        0.063458   
       996           1   0.930453  9.840770e-01        0.053624   
       997           1   1.745980  1.779229e+00        0.033248   
       998           1   0.738298  7.427760e-01        0.004477   
       999           1   2.170938  1.953485e+00       -0.217453   

              taste_shocks_1  taste_shocks_2  value_choice_0  value_choice_1  \
period agent                                                                   
0      0            0.227179       -0.255894             NaN    2.955727e+20   
       1           -0.268403       -0.223158             NaN    2.955727e+20   
       2           -0.257559       -0.246152             NaN    2.955727e+20   
       3           -0.001980       -0.093682             NaN    2.955727e+20   
       4           -0.051862        0.074041             NaN    2.955727e+20   
...                      ...             ...             ...             ...   
19     995         -0.124783       -0.247989        1.258769             NaN   
       996         -0.188307        0.315452        0.984077             NaN   
       997          0.023695       -0.069566        1.779229             NaN   
       998          0.049440        0.211680        0.742776             NaN   
       999         -0.092175       -0.019524        1.953485             NaN   

              value_choice_2  
period agent                  
0      0 